## Etapa 1 - Ingestão
1) Importação das bibliotecas, aqui eu carrego as ferramentas que vou usar ao longo do notebook

In [34]:
from pathlib import Path
import pandas as pd
import sqlite3
import json

2) Definição de caminhos do projeto
 Nesta parte eu organizo a estrutura de pastas de forma programática.
 Por que assim?
 - Fica fácil rodar o notebook em máquinas diferentes sem ter que mudar caminhos na mão.
 - Eu parto da pasta atual (CWD) e descubro a raiz do projeto (PROJECT_DIR).
 - Depois crio caminhos derivados para dados (data/), dado bruto (data/raw/),
   o arquivo do banco (data/netflix.db) e o CSV original.

In [35]:
CWD = Path.cwd().resolve()
PROJECT_DIR = CWD.parent if CWD.name.lower() in {"scr", "src", "notebooks"} else CWD

DATA_DIR = PROJECT_DIR / "data"
RAW_DIR = DATA_DIR / "raw"      
DB_PATH = DATA_DIR / "netflix.db"    
CSV_PATH = RAW_DIR / "netflix_titles.csv"

3) Leitura do CSV e “olhada rápida” no conteúdo
 Antes de ler, eu faço uma checagem com assert para garantir que o arquivo existe.
 Na leitura:
 - low_memory=False: peço para o pandas não adivinhar tipos por pedaços (chunks).
  Isso deixa a inferência de tipos mais consistente (evita colunas "misturadas"),
  mesmo que use um pouco mais de memória.
 Depois imprimo:
 - A dimensão do dataset (linhas, colunas) para ter uma noção de tamanho.
 - As 5 primeiras linhas (amostra).
 - Uma listagem dos nomes de colunas (ajuda a planejar o tratamento).

In [36]:
assert CSV_PATH.exists(), f"Arquivo não encontrado: {CSV_PATH}\n" \
                          f"Coloque 'netflix_titles.csv' em {RAW_DIR}"

df = pd.read_csv(CSV_PATH, low_memory=False)

print("Dimensão do CSV (linhas, colunas):", df.shape)
display(df.head(5))   
display(pd.DataFrame({'colunas': list(df.columns)}))

Dimensão do CSV (linhas, colunas): (8807, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


,colunas
0,show_id
1,type
2,title
3,director
4,cast
5,country
6,date_added
7,release_year
8,rating
9,duration


Observação importante
 Nesta etapa eu estou fazendo ingestão bruta.
 A ideia é primeiro trazer o dado como está para dentro do meu ambiente/banco
 e só depois criar tabelas limpas e normalizadas na etapa de tratamento.


4) Gravação no SQLite (camada bruta)
 Agora eu salvo o dataframe no SQLite.
 Detalhes do que faço aqui:
 - sqlite3.connect(DB_PATH): abre (ou cria) o arquivo do banco netflix.db.
 - to_sql("titles_raw", ...): escrevo a tabela bruta chamada titles_raw.
 - Em seguida, faço uma **validação simples**: conto quantas linhas foram gravadas.

In [37]:
conn = sqlite3.connect(DB_PATH.as_posix())

df.to_sql("titles_raw", conn, if_exists="replace", index=False)

row_count = conn.execute("SELECT COUNT(*) FROM titles_raw").fetchone()[0]
print(f"Linhas gravadas em titles_raw: {row_count}")

Linhas gravadas em titles_raw: 8807


5) Conferindo o schema direto no SQLite
 Eu uso PRAGMA table_info para listar as colunas como o SQLite entendeu (nome, tipo, etc.).
 Isso é útil para:
 - Ver se as colunas foram criadas com os tipos esperados.
 - Planejar as conversões que vou fazer na próxima etapa (ex.: datas, duração, rating).
 Também faço uma consulta de prévia (SELECT * LIMIT 5) direto do banco
 para ter certeza de que a escrita funcionou e que consigo ler de volta via SQL.

In [38]:
schema = conn.execute("PRAGMA table_info(titles_raw)").fetchall()
print("Schema (cid, name, type, notnull, dflt_value, pk):")
for col in schema:
    print(col)

# Prévia consultando direto do banco
preview = pd.read_sql_query("SELECT * FROM titles_raw LIMIT 5", conn)
display(preview)

Schema (cid, name, type, notnull, dflt_value, pk):
(0, 'show_id', 'TEXT', 0, None, 0)
(1, 'type', 'TEXT', 0, None, 0)
(2, 'title', 'TEXT', 0, None, 0)
(3, 'director', 'TEXT', 0, None, 0)
(4, 'cast', 'TEXT', 0, None, 0)
(5, 'country', 'TEXT', 0, None, 0)
(6, 'date_added', 'TEXT', 0, None, 0)
(7, 'release_year', 'INTEGER', 0, None, 0)
(8, 'rating', 'TEXT', 0, None, 0)
(9, 'duration', 'TEXT', 0, None, 0)
(10, 'listed_in', 'TEXT', 0, None, 0)
(11, 'description', 'TEXT', 0, None, 0)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


6) Encerramento
 Fecho a conexão com o banco para liberar o arquivo (`.db`).

In [39]:
conn.close()
print("Ingestão concluída com sucesso.")


Ingestão concluída com sucesso.
